Connect Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import Libraries

In [2]:
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import operator
import tensorflow as tf
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import skimage
from skimage.io import imread, imshow, imsave

In [3]:
def centerCrop(img, bounding):
    start = tuple(map(lambda a, da: a//2-da//2, img.shape, bounding))
    end = tuple(map(operator.add, start, bounding))
    slices = tuple(map(slice, start, end))
    return img[slices]

In [4]:
train_all_path = r'/content/drive/MyDrive/Dataset/C-NMC/High Imbalance/Train - 1 to 100 ratio/all'
train_hem_path = r'/content/drive/MyDrive/Dataset/C-NMC/High Imbalance/Train - 1 to 100 ratio/hem'

In [5]:
train_all_list = os.listdir(train_all_path)
train_all_list.sort()

train_hem_list = os.listdir(train_hem_path)
train_hem_list.sort()



print('train/all: ', len(train_all_list))
print('train/hem :', len(train_hem_list))

train/all:  33
train/hem : 3390


In [6]:
train_all_batch = np.zeros((len(train_all_list), 210, 210, 3), dtype=np.uint8)
train_hem_batch = np.zeros((len(train_hem_list), 210, 210, 3), dtype=np.uint8)


print(train_all_batch.shape, train_hem_batch.shape)

(33, 210, 210, 3) (3390, 210, 210, 3)


In [7]:
def img_crop(list_data, batch, path):
    i=0
    for x in list_data:
        image = imread(os.path.join(path, x))
        image = centerCrop(image, (210,210,3))
        batch[i] = image
        i+=1

    print(type(batch), batch.shape, batch.dtype, batch[0].shape, batch[0].dtype)

In [8]:
img_crop(train_all_list, train_all_batch, train_all_path)
img_crop(train_hem_list, train_hem_batch, train_hem_path)

<class 'numpy.ndarray'> (33, 210, 210, 3) uint8 (210, 210, 3) uint8
<class 'numpy.ndarray'> (3390, 210, 210, 3) uint8 (210, 210, 3) uint8


In [9]:
def save_img(list_data, path, batch):
    for x in range(len(list_data)):
        new_path = path + '/' + list_data[x]
        print(new_path)
        imsave(new_path ,batch[x])

In [ ]:
path = '/content/drive/MyDrive/Dataset/C-NMC/High imbalancec/10x Aug/all'
save_img(train_all_list, path, train_all_batch)
path = '/content/drive/MyDrive/Dataset/C-NMC/High imbalance/10x Aug/hem'
save_img(train_hem_list, path, train_hem_batch)

In [ ]:
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
seq = iaa.Sequential([

        iaa.Fliplr(0.5),
        iaa.Flipud(0.2),

         iaa.Crop(percent=(0, 0.1)),

        sometimes(iaa.CropAndPad(
            pad_mode=ia.ALL,
            percent=(-0.05, 0.1),
            pad_cval=(0, 255)
        )),

      sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
            rotate=(-45, 45),

        )),
],
random_order=True
)

In [ ]:
def AugSave(seq, batch, list_data, path, n):
    images_aug = seq(images=batch)
    print(type(images_aug), images_aug.shape, images_aug.dtype, np.max(images_aug), np.min(images_aug), images_aug[0].shape, images_aug[0].dtype, np.max(images_aug[0]), np.min(images_aug[0]))
    for x in range(len(list_data)):
        new_path = path + '/' + str(n) + 'Aug_' + list_data[x]
        print(new_path)
        imsave(new_path ,images_aug[x])

In [ ]:
times2augment = 10
path = '/content/drive/MyDrive/Dataset/C-NMC/High imbalance/10x Aug/all'
for x in range(times2augment):
    AugSave(seq,train_all_batch, train_all_list, path, x+1)